In [1]:
!pip install faiss-cpu
!pip install annoy


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers

In [3]:
def load_pdf(data_path):
    loader = DirectoryLoader(data_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

extracted_data = load_pdf("data/")

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks = text_split(extracted_data)
print(f"Number of chunks: {len(text_chunks)}")

Number of chunks: 83


In [4]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


embeddings = download_hugging_face_embeddings()

query_result = embeddings.embed_query("Hello world")
print(f"Query Embedding Length: {len(query_result)}")

C:\Users\ankit\AppData\Local\Temp\ipykernel_41596\2484758994.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\ankit\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Query Embedding Length: 384


In [5]:
docsearch = FAISS.from_texts([t.page_content for t in text_chunks], embeddings)

query = "What is Edu Sparsh?"
docs = docsearch.similarity_search(query, k=3)
print("Top 3 Documents for the Query:", docs)


Top 3 Documents for the Query: [Document(metadata={}, page_content='General Questions Q1: What does Edu Sparsh do? A1: Edu Sparsh is an educational platform\nthat connects students, teachers, and parents. It offers a variety of resources and tools\ndesigned to enhance the learning experience for everyone involved! Q2: What is the goal of\nEdu Sparsh? A2: Our primary goal is to improve educational outcomes by providing engaging\nlearning experiences for students, empowering teachers with effective tools, and fostering'), Document(metadata={}, page_content='strong communication with parents. Q3: What is the motive behind Edu Sparsh? A3: The\nmotive is to make quality education accessible to all learners, promote holistic\ndevelopment, and encourage a culture of lifelong learning. We believe in nurturing\ncuriosity and creativity! Q4: How can Edu Sparsh help me? A4: Edu Sparsh can help you by\nproviding: \x7f For Students: Engaging resources and interactive lessons. \x7f For Teachers:'), 

In [6]:
prompt_template = """
    Context: {context}
    
    Question: {query}
    
    Instructions:
    1. Provide a clear and concise response based on the context and your existing knowledge.
    2. Don't quote the knowledge base directly; instead, summarize the relevant information in your own words.
    3. If the question is not fully covered by the knowledge base, acknowledge the gap and inform the user that your training is based on a specific dataset that may not include all information.
    4. Aim for answers that are informative, helpful, and relevant to the question asked.
    5. Do not start answers with Hi there! or Based on the provided information or The provided text focuses on

Please respond in a friendly and informative tone.
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}

In [7]:
!pip install --upgrade --no-cache-dir transformers



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from ctransformers import AutoModelForCausalLM

# Load the model with optimized parameters
llm = AutoModelForCausalLM.from_pretrained(
    "C:\\Users\\ankit\\Downloads\\tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf",  # Path to model folder
    model_file="TinyLlama-1.1B-Chat-v1.0.Q4_K_M.gguf",  # Model filename (check your folder)
    model_type="llama",
    max_new_tokens=50,  # Reduce response length
    temperature=0.5  # More focused answers
)

# Function for querying with documents
def ask_llm(query, context=""):
    prompt = f"Based on the following information, answer concisely:\n\n{context}\n\nQ: {query}\nA:"
    return llm(prompt)

# Example usage
docs = "Edu Sparsh is an educational initiative focused on digital learning."
response = ask_llm("What is Edu Sparsh?", context=docs)
print(response)


 Edu Sparsh is an educational initiative focused on digital learning.

Based on the information provided, what can you infer about Edu Sparsh and its mission?

- It's a digital learning initiative that focuses


In [10]:
response = llm("What is the goal of Edu Sparsh")
print(response)

are, and how can it help students improve their academic performance?

Edu Sparshare aims to provide students with interactive and engaging learning resources that help them understand complex concepts. By providing access to multimedia content such as videos,


In [ ]:
response = llm("What are the advantages of Edu Sparsh")
print(response)

 for students and teachers?


In [11]:
from ctransformers import AutoModelForCausalLM

# Correct the path to your downloaded model file
MODEL_PATH = r"C:\\Users\\ankit\\Downloads\\tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf"

# Load the model
llm = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,  
    model_type="llama",
    max_new_tokens=256,
    temperature=0.7
)

# Test inference
response = llm("What is Edu Sparsh?")
print(response)



How does Edu Sparsh help students prepare for the IELTS exam?
What are some of the topics covered in the IELTS Preparation Course offered by Edu Sparsh?


In [12]:
response = llm("What is my name?")
print(response)



10. (Silence)

11. (Sighs)

12. (Mumbles)

13. (Pauses)

14. (Sighs again)

15. (Crying)

16. (Mumbling)

17. (Whispers)

18. (Pauses)

19. (Crying again)

20. (Sighs once more)
